In [19]:
import pandas as pd
import numpy as np
from gurobipy import *
import random
import itertools
import time 


In [20]:

# Parameters
teams = ["Adelaide Crows", 
         "Brisbane Lions", 
         "Carlton Blues", 
         "Collingwood Magpies",
         "Essendon Bombers", 
         "Fremantle Dockers", 
         "Geelong Cats", 
         "Gold Coast Suns",
         "Greater Western Sydney Giants", 
         "Hawthorn Hawks", 
         "Melbourne Demons",
         "North Melbourne Kangaroos", 
         "Port Adelaide Power",
         "Richmond Tigers",
         "St Kilda Saints",
         "Sydney Swans",
         "West Coast Eagles",
         "Western Bulldogs"]

team_numbers = {team: number for number, team in enumerate(sorted(teams), start=0)}

locations = ['VIC','NSW','SA','WA','QLD']

location_numbers = {location: number for number, location in enumerate(sorted(locations), start=0)}

home_locations = ['SA','QLD','VIC','VIC','VIC','WA','VIC','QLD','NSW','VIC',
                          'VIC','VIC','SA','VIC','VIC','NSW','WA','VIC']

ranking = [10,2,5,1,11,14,12,15,7,16,4,17,3,13,6,8,18,9] # 2023 pre-finals rankings

team_fans = [60,45,93,102,79,61,81,20,33,72,68,50,60,100,58,63,100,55] # 2023 number of members, in 000's

wins = [11,17,13.5,18,11,10,10.5,9,13,7,16,3,17,10.5,13,12.5,3,12]

stadiums = ['MCG','Marvel','GMHBA','Adelaide Oval','Optus','Gabba','HBS','SCG','Giants']

stadium_numbers = {stadium: number for number, stadium in enumerate(sorted(stadiums), start=0)}

stadium_locations = ['VIC','VIC','VIC','SA','WA','QLD','QLD','NSW','NSW']

home_stadiums = [['Adelaide Oval'],['Gabba'],['MCG','Marvel'],['MCG','Marvel'],['MCG','Marvel'],['Optus'],['GMHBA'],
                 ['HBS'],['Giants'],['MCG'],['MCG'],['Marvel'],['Adelaide Oval'],
                 ['MCG'],['Marvel'],['SCG'],['Optus'],['Marvel']]

home_location_stadiums = [[] for i in range(len(teams))]
for i in range(len(teams)):
    for j in range(len(stadiums)):
        if stadium_locations[j] == home_locations[i]:
            home_location_stadiums[i].append(stadiums[j])

stadium_size = [100,53,40,54,60,39,27,47,24] # Stadium sizes in 000's

rivals = [['Port Adelaide Power'],
          ['Gold Coast Suns','Collingwood Magpies'], # From wikipedia
          ['Essendon Bombers','Richmond Tigers','Collingwood Magpies', 'Fremantle Dockers'],
          ['Carlton Blues','Essendon Bombers','Brisbane Lions','Melbourne Demons','Richmond Tigers','Geelong Cats',
           'Hawthorn Hawks', 'West Coast Eagles','Port Adelaide Power'],
          ['Carlton Blues','Collingwood Magpies','Richmond Tigers','Hawthorn Hawks','North Melbourne Kangaroos'],
          ['Carlton Blues','West Coast Eagles'],
          ['Collingwood Magpies','Hawthorn Hawks'],
          ['Brisbane Lions'],
          ['Sydney Swans','Western Bulldogs'],
          ['Hawthorn Hawks','Geelong Cats','Essendon Bombers', 'North Melbourne Kangaroos'],
          ['Collingwood Magpies'],
          ['Essendon Bombers','Hawthorn Hawks'],
          ['Adelaide Crows','Collingwood Magpies'],
          ['Carlton Blues','Collingwood Magpies','Essendon Bombers'],
          ['Sydney Swans'],
          ['West Coast Eagles','St Kilda Saints','Greater Western Sydney Giants','Hawthorn Hawks'],
          ['Fremantle Dockers','Sydney Swans','Collingwood Magpies'],
          ['Greater Western Sydney Giants']]

rivals_num = [[team_numbers[i] for i in rivals[j]] for j in range(len(rivals))]

timeslots = [i for i in range(7)]
timeslot_values = [10,13,5,6,11,5,4] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening']

Ts = range(len(teams))
Ss = range(len(stadiums))
timeslots = range(7)
rounds = range(22)

# Decision Variables
fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]

In [21]:
# Daniel's Feasibility Function

def feasibility(fixture):
    violated = 0
    critical = 0
    
    for i in Ts: # Each team plays once a week
        for r in rounds:
            critical += abs(sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for j in Ts for s in Ss for t in timeslots)-1)
            
    
    for i in Ts:
        critical += abs(sum(fixture[r][t][stadium_numbers[s]][j][i] for j in Ts for s in home_stadiums[i] 
                                 for t in timeslots for r in rounds)-11)
    
    
    for i in Ts:
        critical += sum(fixture[r][t][s][i][i] for s in Ss for t in timeslots for r in rounds)
        
        for j in Ts:
            if i != j:
                violated += max(sum(fixture[r][t][s][j][i] for s in Ss for t in timeslots for r in rounds)-1,0)
                violated += max(1-sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for s in Ss for t in timeslots for r in rounds),0)
                
                
    for i in Ts:
        for r in rounds[:-1]:
            violated += max(0,sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for j in Ts for s in Ss for t in [5,6])+ 
                            sum(fixture[r+1][t][s][j][i]+fixture[r+1][t][s][i][j] for j in Ts for s in Ss for t in [0]) - 1)
            
    
    # Three games in a row outside home location
    for i in Ts:
        for r in rounds[:-2]:
            violated += max(0,1-sum(fixture[r_][t][stadium_numbers[s]][i][j]+fixture[r_][t][stadium_numbers[s]][j][i] for j in Ts for s in home_location_stadiums[i] 
                                 for t in timeslots for r_ in range(r,r+3)))
       
    # Four away games in a row
    for i in Ts:
        for r in rounds[:-3]:
            violated += max(0,1-sum(fixture[r_][t][s][j][i] for j in Ts for s in Ss for t in timeslots for r_ in range(r,r+4)))
    
    
    # Constraint 7: 2+ games in one day in the same stadium
    for r in rounds:
        for s in Ss:
            
            violated += max(0,sum(fixture[r][t][s][j][i] for i in Ts for j in Ts for t in [5, 6])-1)
            
            violated += max(0,sum(fixture[r][t][s][j][i] for i in Ts for j in Ts for t in [2, 3, 4])-1)
            
            for t in [0,1]:
                violated += max(0,sum(fixture[r][t][s][j][i] for i in Ts for j in Ts)-1)
    
    
    # Constraint: No more than two games in any timeslot, and only one on Thursday and Friday night, at least one in each
    for r in rounds:
        
        for t in [2,3,4,5,6]:
            violated += max(0,1-sum(fixture[r][t][s][j][i] for i in Ts for j in Ts for s in Ss)) # At least one game each timeslot
            violated += max(0,sum(fixture[r][t][s][j][i] for i in Ts for j in Ts for s in Ss)-2)
        
        for t in [0,1]:
            #Changed this from 'abs' to 'max'
            violated += max(0,sum(fixture[r][t][s][j][i] for i in Ts for j in Ts for s in Ss)-1) # One game
            
            
    return violated, critical

In [22]:
# Attractiveness Function Parameters, adjust them as needed
alpha = 1.0
beta = 1.0
gamma = 1.0
sigma = 1.0
xi = 1.0

def attractiveness(i, j, s, t, r):
    score = 1
    if r == 0:
        score *= 4
    elif r == 1:
        score *= 2
    elif r == 21:
        score *= 1.5
    
    if j in [team_numbers[i] for i in rivals[0]]:
        score *= 1+alpha
    
    score /= np.sqrt(1+abs(ranking[i]-ranking[j]))
    score  /= np.sqrt(ranking[i]+ranking[j])
    
    if stadium_locations[s] == home_locations[j]:
        score *= (1+beta)
        
    score *= np.sqrt(stadium_size[s])
    score *= (team_fans[i]+0.5*team_fans[j])
    
    score *= timeslot_values[t]
    
    return score

In [23]:
def probability_win(i, j, s):
    probability = wins[i]/(wins[i]+wins[j])
    if stadiums[s] not in home_location_stadiums[j]:
        probability += (1-probability)/2.5
    elif stadiums[s] not in home_stadiums[j]:
        probability += (1-probability)/4
    else:
        probability += (1-probability)/10
        
    return probability

In [24]:
def expected_win_variance(fixture):
    results = []
    expected_wins = [0]*18
    for r in rounds:
        for i in Ts:
            for j in Ts:
                for s in Ss:
                    for t in timeslots:
                        expected_wins[i] += probability_win(i, j, s)*fixture[r][t][s][j][i]
                        expected_wins[i] += (1-probability_win(j, i, s))*fixture[r][t][s][i][j]
        
        results.append(np.var(expected_wins))
    
    # print(results)
    return sum((i+1)*results[i] for i in range(len(results)))

In [25]:
def expected_win_variance(fixture):
    results = []
    expected_wins = [0]*18
    for r in rounds:
        for i in Ts:
            for j in Ts:
                for s in Ss:
                    for t in timeslots:
                        expected_wins[i] += probability_win(i, j, s)*fixture[r][t][s][j][i]
                        expected_wins[i] += (1-probability_win(j, i, s))*fixture[r][t][s][i][j]
        
        results.append(np.var(expected_wins))
    
    # print(results)
    return sum((i+1)*results[i] for i in range(len(results)))

In [26]:
def fixture_attractiveness(fixture,max_value,violated_factor,critical_factor,equality_factor):
    total_score = 0
    
    for r in rounds:
        for t in timeslots:
            value = 0
            for i in Ts:
                for j in Ts:
                    for s in Ss:
                        value += attractiveness(i, j, s, t, r)*fixture[r][t][s][j][i]
            
            total_score += min(max_value,value)

    violated, critical = feasibility(fixture)
    equality = equality_factor*expected_win_variance(fixture)

    # print(total_score)
    # print('-', violated_factor*violated)
    # print('-', critical_factor*critical)
    # print('-', equality)
    
    return total_score - violated_factor*violated - critical_factor*critical - equality


In [27]:
def objective_value(fixture):

    violated, critical = feasibility(fixture)

    violated_penalty = 2*10^4
    critical_penalty = 10^6
    # for i in Ts:
    #     for j in Ts:
    #         for s in Ss:
    #             for r in rounds:
    #                 for t in timeslots:
    #                     print(attractiveness(i,j,s,t,r))

    objective_value = - violated_penalty*violated - critical_penalty*critical + fixture_attractiveness(fixture, 2*10^4, critical_penalty, violated_penalty, 10^3) 

    return objective_value

In [28]:
def genesis(pop_size=5):
    print('Genesis')

    # MIP SOLUTION
    MIP_soln = np.load('solutions/mip_initial_fixture.npy')
    
    pop = []
    pop.append([MIP_soln, objective_value(MIP_soln)])

    for i in range(pop_size-1):
        fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]   
        available_timeslots = list(timeslots)
        available_multiple_game_timeslots = [2,3,4,5,6]    
        for r in rounds: 
            for game in range(9):
                # Randomly choose two teams. Ensure there is at least one rivalry match. 
                if game == 0:
                    selection_i = random.choice(Ts)
                    selection_j = random.choice(rivals_num[selection_i])
                    selection_stadium = random.choice(home_location_stadiums[selection_i])
                    selection_stadium = stadium_numbers[selection_stadium]
                    if len(available_timeslots) > 0:  
                        t = random.choice(available_timeslots)
                        available_timeslots.remove(t)
                    else: 
                        t = random.choice(available_multiple_game_timeslots)
                        available_multiple_game_timeslots.remove(t)
                    fixture_matrix[r][t][selection_stadium][selection_j][selection_i] = 1 
                    # print('Rivalry Round: ', teams[selection_i], ' vs. ', teams[selection_j])
                else: 
                    team_pairing = random.sample(Ts, 2)
                    selection_i = team_pairing[0]
                    selection_j = team_pairing[1]
                    selection_stadium = random.choice(home_location_stadiums[selection_i])
                    selection_stadium = stadium_numbers[selection_stadium]
                    if len(available_timeslots) > 0:  
                        t = random.choice(available_timeslots)
                        available_timeslots.remove(t)
                    else: 
                        t = random.choice(multiple_game_timeslots)
                    fixture_matrix[r][t][selection_stadium][selection_j][selection_i] = 1 
                    # print(teams[selection_i], ' vs. ', teams[selection_j])    
            # print('\n')  
        pop.append([fixture_matrix, objective_value(fixture_matrix)])
    return pop

In [29]:
import random

def select_parents(population):
    """
    Select two individuals from a population for reproduction.
    
    Args:
    - population (list): A list of individuals, where each individual is represented as a tuple (individual_data, objective_value).
    
    Returns:
    - parent1 (tuple): The first parent selected.
    - parent2 (tuple): The second parent selected.
    """
    
    print('Romance')
    # Sort the population by objective value in descending order (higher is better)
    sorted_population = sorted(population, key=lambda x: x[1], reverse=True)

    # Calculate the number of elite individuals (top 25%)
    num_elite = len(sorted_population) // 4

    if num_elite > 1:
        # If there are enough elite individuals, select the first parent as an elite individual
        parent1 = random.choice(sorted_population[:num_elite])
    else:
        # If there are not enough elite individuals, choose both parents randomly
        parent1 = random.choice(sorted_population)
    
    # Select the second parent randomly from the entire population
    parent2 = random.choice(population)

    return parent1, parent2



In [30]:
def get_dimensions(lst):
    dimensions = []

    while isinstance(lst, list):
        dimensions.append(len(lst))
        lst = lst[0] if len(lst) > 0 else None

    return dimensions

In [31]:
def birth(parent1, parent2):
    print('Birth')
    # print(get_dimensions(parent1), get_dimensions(parent2))
    # Determine a random crossover point
    crossover_point = np.random.randint(0, len(parent1))  # Include 0, exclude len(parent1)

    # Create the first child by combining the first part of parent1 and the second part of parent2
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

    # Create the second child by combining the first part of parent2 and the second part of parent1
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    return [child1, objective_value(child1)], [child2, objective_value(child2)]
    

In [32]:
def evolutionPlus():

    start_time = time.time()  # Record the start time

    # Parameters 
    pop_size = 5  # Adjust as needed
    solns = []
    num_gen = 0
    duration = 60 * 10

    pop = genesis(pop_size)  # Use the provided pop_size when calling genesis

    best_soln = max(pop, key=lambda x: x[1])
    while time.time() - start_time < duration:
        num_gen += 1
        # Select Parents
        parent1, parent2 = select_parents(pop)

        # Create Children
        child1, child2 = birth(parent1[0], parent2[0])
        pop.append(child1)
        pop.append(child2)
        
        # Death
        family = [parent1, parent2, child1, child2]
        # Find family member with the minimum objective value
        min_value = 1000000000000
        to_die_idx = None
        for i, member in enumerate(family):
            if member[1] < min_value:
                min_value = member[1]
                to_die_idx = i

        # Remove the element at to_die_idx
        if to_die_idx is not None:
            print('Death')
            pop.pop(to_die_idx)

        # Occassionally perform local optimisation

        # Occasionally perform immigration

        # Find the index of the element with the largest objective value
        max_index = 0
        max_value = pop[0][1]
        for i, soln in enumerate(pop):
            if soln[1] > max_value:
                max_value = soln[1]
                max_index = i

        # Extract the element with the largest objective value 
        best_soln = pop[max_index]
        solns.append(best_soln)

    for i, soln in enumerate(pop):
        if soln[1] > max_value:
            max_index = i
    best_soln = pop[max_index]

    print('Solution: ', best_soln[0])
    print('Value: ', best_soln[1])

    return best_soln


In [33]:
best_soln = evolutionPlus()

Genesis


NameError: name 'multiple_game_timeslots' is not defined

In [ ]:
best_fixture = best_soln[0]
for r in rounds: 
    print('\nRound ', r)
    for t in timeslots:
        for i in Ts:
            for j in Ts:
                for s in range(len(Ss)-1):
                    if best_fixture[r][t][s][j][i] == 1:
                        if j in [team_numbers[rival] for rival in rivals[i]]:
                            print("Rivalry Match! ", teams[i], " VS ", teams[j], ' AT ', stadiums[s], ' ON ', timeslot_names[t])
                        else: 
                            print(teams[i], " VS ", teams[j], ' AT ', stadiums[s], ' ON ', timeslot_names[t])


Round  0
Adelaide Crows  VS  Richmond Tigers  AT  MCG  ON  Friday Night
Carlton Blues  VS  Brisbane Lions  AT  Gabba  ON  Friday Night
Rivalry Match!  Collingwood Magpies  VS  West Coast Eagles  AT  Gabba  ON  Friday Night
Essendon Bombers  VS  Western Bulldogs  AT  Gabba  ON  Friday Night
Fremantle Dockers  VS  Sydney Swans  AT  SCG  ON  Friday Night
Rivalry Match!  Geelong Cats  VS  Hawthorn Hawks  AT  Marvel  ON  Friday Night
Greater Western Sydney Giants  VS  St Kilda Saints  AT  Adelaide Oval  ON  Friday Night
Melbourne Demons  VS  Gold Coast Suns  AT  Gabba  ON  Friday Night
North Melbourne Kangaroos  VS  Port Adelaide Power  AT  HBS  ON  Friday Night

Round  1
Essendon Bombers  VS  Geelong Cats  AT  HBS  ON  Saturday Afternoon
Port Adelaide Power  VS  West Coast Eagles  AT  MCG  ON  Saturday Afternoon
Western Bulldogs  VS  Port Adelaide Power  AT  Marvel  ON  Saturday Evening
Rivalry Match!  Essendon Bombers  VS  Carlton Blues  AT  Marvel  ON  Saturday Night
West Coast Eagles  